In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc, balanced_accuracy_score, f1_score, precision_recall_curve
from sklearn.calibration import CalibratedClassifierCV
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
import seaborn as sns
import os

# --- Etapa 0: Configuração ---
# Criar diretório para salvar os gráficos, se não existir
output_dir = '/kaggle/working/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Configurar o estilo dos gráficos para melhor visualização
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14

# --- Etapa 1: Carregamento e Pré-processamento dos Dados ---

# Carregar o dataset
try:
    df = pd.read_csv('/kaggle/input/dataset-clean/dados_limpos.csv')
    print("Dataset carregado do caminho Kaggle.")
except FileNotFoundError:
    try:
        df = pd.read_csv('dados_limpos.csv') 
        print("Dataset carregado localmente.")
    except FileNotFoundError:
        print("Arquivo 'dados_limpos.csv' não encontrado no caminho Kaggle nem localmente.")
        print("Por favor, certifique-se de que o arquivo está no diretório correto ou ajuste o caminho.")
        exit()

# Selecionar colunas relevantes - Expandindo para mais features
colunas_features = [
    'Nível de ensino alcançado', 
    'Tempo de experiência na área de dados',
    'Área de formação acadêmica',  # Nova feature
    'Nível de senioridade',        # Nova feature
    'UF onde mora',                # Nova feature
    'Setor de atuação da empresa'  # Nova feature
]
coluna_target = 'Faixa salarial mensal'
colunas_necessarias = colunas_features + [coluna_target]

# Remover linhas com valores ausentes nas colunas cruciais
df_limpo = df[colunas_necessarias].copy()
df_limpo.dropna(subset=colunas_necessarias, inplace=True)

# --- Etapa 2: Engenharia de Features e Criação da Variável Alvo ---

# Mapeamento ordinal para 'Nível de ensino alcançado'
nivel_ensino_map = {
    'Estudante de Graduação': 0,
    'Graduação/Bacharelado': 1,
    'Pós-graduação': 2,
    'Mestrado': 3,
    'Doutorado ou Phd': 4
}
df_limpo['formacao_academica_encoded'] = df_limpo['Nível de ensino alcançado'].map(nivel_ensino_map)

# Mapeamento ordinal para 'Tempo de experiência na área de dados'
experiencia_map = {
    'Menos de 1 ano': 0,
    'de 1 a 2 anos': 1,
    'de 3 a 4 anos': 2,
    'de 4 a 6 anos': 3,
    'de 5 a 6 anos': 3, 
    'de 7 a 10 anos': 4,
    'Mais de 10 anos': 5
}
df_limpo['experiencia_profissional_encoded'] = df_limpo['Tempo de experiência na área de dados'].map(experiencia_map)

# Mapeamento ordinal para 'Nível de senioridade'
senioridade_map = {
    'Júnior': 0,
    'Pleno': 1,
    'Sênior': 2
}
df_limpo['senioridade_encoded'] = df_limpo['Nível de senioridade'].map(senioridade_map)

# Mapeamento ordinal para 'Faixa salarial mensal' para criar a variável alvo binária
salario_map_ordinal = {
    'Menos de R$ 1.000/mês': 0,
    'de R$ 1.001/mês a R$ 2.000/mês': 1,
    'de R$ 2.001/mês a R$ 3.000/mês': 2,
    'de R$ 3.001/mês a R$ 4.000/mês': 3,
    'de R$ 4.001/mês a R$ 6.000/mês': 4,
    'de R$ 6.001/mês a R$ 8.000/mês': 5,
    'de R$ 8.001/mês a R$ 12.000/mês': 6,
    'de R$ 12.001/mês a R$ 16.000/mês': 7,
    'de R$ 16.001/mês a R$ 20.000/mês': 8,
    'de R$ 20.001/mês a R$ 25.000/mês': 9,
    'de R$ 25.001/mês a R$ 30.000/mês': 10,
    'de R$ 30.001/mês a R$ 40.000/mês': 11,
    'Acima de R$ 40.001/mês': 12
}
df_limpo['faixa_salarial_encoded'] = df_limpo['Faixa salarial mensal'].map(salario_map_ordinal)

# Criar variável alvo binária: 0 para salários até R$ 8.000 (<=5), 1 para salários acima (>5)
df_limpo['salario_alto'] = df_limpo['faixa_salarial_encoded'].apply(lambda x: 1 if x > 5 else 0)

# Codificação one-hot para variáveis categóricas
df_encoded = pd.get_dummies(df_limpo, columns=['Área de formação acadêmica', 'UF onde mora', 'Setor de atuação da empresa'])

# Remover NaNs que podem surgir de mapeamentos incompletos
df_encoded.dropna(subset=['formacao_academica_encoded', 'experiencia_profissional_encoded', 
                         'senioridade_encoded', 'faixa_salarial_encoded'], inplace=True)

# Selecionar colunas para o modelo (excluindo as originais não codificadas)
X_columns = ['formacao_academica_encoded', 'experiencia_profissional_encoded', 'senioridade_encoded'] + \
           [col for col in df_encoded.columns if col.startswith(('Área de formação acadêmica_', 
                                                               'UF onde mora_', 
                                                               'Setor de atuação da empresa_'))]
X = df_encoded[X_columns]
y = df_encoded['salario_alto']

# Verificar se há dados suficientes
if X.shape[0] < 10 or len(y.unique()) < 2:
    print("Não há dados suficientes ou classes suficientes após o pré-processamento para treinar o modelo.")
    print(f"Tamanho de X: {X.shape}, Classes em y: {y.unique()}")
    exit()

# Verificar o balanceamento das classes
class_counts = y.value_counts()
print("\nDistribuição das classes:")
print(f"Salário Baixo/Médio (0): {class_counts[0]} ({class_counts[0]/len(y)*100:.2f}%)")
print(f"Salário Alto (1): {class_counts[1]} ({class_counts[1]/len(y)*100:.2f}%)")

# --- Etapa 3: Balanceamento dos Dados ---

# Alternativa ao SMOTE: Usar class_weight e sample_weight
# Calculando os pesos para cada amostra com base na classe
class_weights = {0: 1.0, 1: class_counts[0] / class_counts[1]}
sample_weights = np.array([class_weights[cls] for cls in y])

# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test, sample_weights_train, _ = train_test_split(
    X, y, sample_weights, test_size=0.3, random_state=42, stratify=y
)

print("\nTamanho dos conjuntos de dados:")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

# --- Etapa 4: Desenvolvimento do Modelo de Machine Learning - Random Forest ---

# Definir a grade de parâmetros para o GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [3, 5, 7],
    'class_weight': ['balanced', 'balanced_subsample']
}

# Instanciar o RandomForestClassifier
rf_base = RandomForestClassifier(random_state=42, n_jobs=-1)

# Instanciar o GridSearchCV com balanced_accuracy_score como métrica
balanced_acc_scorer = 'balanced_accuracy'  # Usar métrica balanceada em vez de acurácia simples

# Instanciar o GridSearchCV
grid_search = GridSearchCV(estimator=rf_base, param_grid=param_grid,
                          cv=5, n_jobs=-1, verbose=1, scoring=balanced_acc_scorer)

print("Iniciando a busca de hiperparâmetros com GridSearchCV...")
grid_search.fit(X_train, y_train, sample_weight=sample_weights_train)

# Melhor modelo encontrado pelo GridSearchCV
best_rf_model = grid_search.best_estimator_

print("\nMelhores Parâmetros Encontrados pelo GridSearchCV:")
print(grid_search.best_params_)

# --- Etapa 5: Calibração do Modelo ---

# Calibrar as probabilidades do modelo para obter estimativas mais confiáveis
calibrated_model = CalibratedClassifierCV(
    base_estimator=best_rf_model,
    method='isotonic',  # ou 'sigmoid'
    cv=5
)

calibrated_model.fit(X_train, y_train, sample_weight=sample_weights_train)

# --- Etapa 6: Avaliação com Diferentes Limiares ---

# Previsões com o modelo calibrado
y_pred_train = calibrated_model.predict(X_train)
y_pred_proba_test = calibrated_model.predict_proba(X_test)[:, 1]  # Probabilidades para classe positiva

# Testar diferentes limiares
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
results = []

print("\nAvaliação com diferentes limiares de classificação:")
for threshold in thresholds:
    y_pred_custom = (y_pred_proba_test >= threshold).astype(int)
    
    # Calcular métricas
    acc = accuracy_score(y_test, y_pred_custom)
    bal_acc = balanced_accuracy_score(y_test, y_pred_custom)
    f1 = f1_score(y_test, y_pred_custom)
    
    # Matriz de confusão
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_custom).ravel()
    
    # Calcular precisão e recall para cada classe
    precision_0 = tn / (tn + fn) if (tn + fn) > 0 else 0
    recall_0 = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision_1 = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_1 = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    results.append({
        'threshold': threshold,
        'accuracy': acc,
        'balanced_accuracy': bal_acc,
        'f1_score': f1,
        'precision_0': precision_0,
        'recall_0': recall_0,
        'precision_1': precision_1,
        'recall_1': recall_1
    })
    
    print(f"\nLimiar: {threshold}")
    print(f"Acurácia: {acc:.4f}")
    print(f"Acurácia Balanceada: {bal_acc:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Matriz de Confusão:")
    print(f"TN: {tn}, FP: {fp}")
    print(f"FN: {fn}, TP: {tp}")
    print(f"Precisão (Classe 0): {precision_0:.4f}, Recall (Classe 0): {recall_0:.4f}")
    print(f"Precisão (Classe 1): {precision_1:.4f}, Recall (Classe 1): {recall_1:.4f}")

# Encontrar o melhor limiar com base na acurácia balanceada
best_threshold_idx = max(range(len(results)), key=lambda i: results[i]['balanced_accuracy'])
best_threshold = results[best_threshold_idx]['threshold']

print(f"\nMelhor limiar encontrado: {best_threshold} (Acurácia Balanceada: {results[best_threshold_idx]['balanced_accuracy']:.4f})")

# Usar o melhor limiar para as predições finais
y_pred_final = (y_pred_proba_test >= best_threshold).astype(int)

# --- Etapa 7: Avaliação Final do Modelo ---

print("\nRelatório de Classificação Final (com limiar otimizado):")
print(classification_report(y_test, y_pred_final, target_names=['Salário Baixo/Médio', 'Salário Alto']))

# --- Etapa 8: Geração de Gráficos ---

# 8.1. Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_final)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=['Salário Baixo/Médio', 'Salário Alto'],
           yticklabels=['Salário Baixo/Médio', 'Salário Alto'],
           annot_kws={"size": 14})
plt.title('Matriz de Confusão (Conjunto de Teste - Limiar Otimizado)', fontsize=16)
plt.xlabel('Previsto', fontsize=14)
plt.ylabel('Verdadeiro', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'matriz_confusao_otimizada.png'), bbox_inches='tight', dpi=300)
plt.close()

# 8.2. Curva ROC
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_test)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 8))
plt.plot(fpr, tpr, color='darkorange', lw=3, label=f'Curva ROC (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.axvline(x=fpr[np.argmin(np.abs(tpr - best_threshold))], color='green', linestyle='--', 
           label=f'Limiar Ótimo = {best_threshold}')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (FPR)', fontsize=14)
plt.ylabel('Taxa de Verdadeiros Positivos (TPR)', fontsize=14)
plt.title('Curva ROC com Limiar Otimizado', fontsize=16)
plt.legend(loc="lower right", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'curva_roc_otimizada.png'), bbox_inches='tight', dpi=300)
plt.close()

# 8.3. Curva Precision-Recall
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_pred_proba_test)

plt.figure(figsize=(10, 8))
plt.plot(recall, precision, color='blue', lw=3)
plt.xlabel('Recall', fontsize=14)
plt.ylabel('Precision', fontsize=14)
plt.title('Curva Precision-Recall', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'precision_recall_curve.png'), bbox_inches='tight', dpi=300)
plt.close()

# 8.4. Importância das Features - MELHORADA
importances = best_rf_model.feature_importances_
feature_names = X.columns
indices = np.argsort(importances)[::-1]

# Limitar para as 20 features mais importantes para melhor visualização
n_features_to_show = min(20, len(indices))
top_indices = indices[:n_features_to_show]

plt.figure(figsize=(16, 10))
plt.title('Importância das 20 Features Mais Relevantes (Random Forest)', fontsize=16)
plt.barh(range(n_features_to_show), importances[top_indices], align='center', color='#1f77b4')
plt.yticks(range(n_features_to_show), [feature_names[i] for i in top_indices], fontsize=12)
plt.xlabel('Importância Relativa', fontsize=14)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'importancia_features_top20.png'), bbox_inches='tight', dpi=300)
plt.close()

# Versão interativa para exploração completa de todas as features
if len(indices) > 20:
    # Criar um gráfico separado para todas as features, organizado por grupos
    # Agrupar features por prefixo para melhor organização
    prefixes = {}
    for i in indices:
        feature = feature_names[i]
        prefix = feature.split('_')[0] if '_' in feature else feature
        if prefix not in prefixes:
            prefixes[prefix] = []
        prefixes[prefix].append((i, importances[i]))
    
    # Plotar gráficos separados por grupo
    for prefix, features in prefixes.items():
        if len(features) > 0:
            # Ordenar features por importância
            features.sort(key=lambda x: x[1], reverse=True)
            
            plt.figure(figsize=(12, max(6, len(features) * 0.4)))
            plt.title(f'Importância das Features: Grupo {prefix}', fontsize=16)
            plt.barh(range(len(features)), [imp for _, imp in features], align='center', color='#2ca02c')
            plt.yticks(range(len(features)), [feature_names[i] for i, _ in features], fontsize=11)
            plt.xlabel('Importância Relativa', fontsize=14)
            plt.grid(axis='x', linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.savefig(os.path.join(output_dir, f'importancia_features_grupo_{prefix}.png'), 
                       bbox_inches='tight', dpi=300)
            plt.close()

# 8.5. Distribuição das Probabilidades Preditas
plt.figure(figsize=(12, 8))
sns.histplot(y_pred_proba_test, bins=50, kde=True)
plt.axvline(x=best_threshold, color='red', linestyle='--', linewidth=2,
           label=f'Limiar Ótimo = {best_threshold}')
plt.title('Distribuição das Probabilidades Preditas', fontsize=16)
plt.xlabel('Probabilidade de Salário Alto', fontsize=14)
plt.ylabel('Contagem', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'distribuicao_probabilidades.png'), bbox_inches='tight', dpi=300)
plt.close()

# 8.6. Visualizar uma árvore do modelo - MELHORADA
plt.figure(figsize=(24, 18))  # Aumentar significativamente o tamanho
plot_tree(best_rf_model.estimators_[0], 
          feature_names=X.columns, 
          class_names=['Salário Baixo/Médio', 'Salário Alto'],
          filled=True, 
          rounded=True, 
          fontsize=12,  # Aumentar o tamanho da fonte
          max_depth=4)  # Aumentar a profundidade para mostrar mais detalhes
plt.title('Visualização de uma Árvore do Random Forest', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'arvore_exemplo_melhorada.png'), 
           bbox_inches='tight', dpi=300)  # Aumentar a resolução
plt.close()

# Versão simplificada da árvore para melhor interpretabilidade
plt.figure(figsize=(20, 12))
plot_tree(best_rf_model.estimators_[0], 
          feature_names=X.columns, 
          class_names=['Salário Baixo/Médio', 'Salário Alto'],
          filled=True, 
          rounded=True, 
          fontsize=12,
          max_depth=3)  # Limitar a profundidade para maior clareza
plt.title('Visualização Simplificada de uma Árvore do Random Forest', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'arvore_exemplo_simplificada.png'), 
           bbox_inches='tight', dpi=300)
plt.close()

# 8.7. Análise de Interação entre Formação e Experiência - MELHORADA
plt.figure(figsize=(14, 10))
pivot_table = pd.crosstab(
    index=df_limpo['formacao_academica_encoded'], 
    columns=df_limpo['experiencia_profissional_encoded'],
    values=df_limpo['salario_alto'],
    aggfunc=np.mean
)

# Renomear índices e colunas para melhor interpretação
formacao_labels = {v: k for k, v in nivel_ensino_map.items()}
experiencia_labels = {v: k for k, v in experiencia_map.items()}

pivot_table.index = [formacao_labels.get(i, i) for i in pivot_table.index]
pivot_table.columns = [experiencia_labels.get(i, i) for i in pivot_table.columns]

# Usar uma paleta de cores mais contrastante e adicionar anotações maiores
sns.heatmap(pivot_table, annot=True, cmap='viridis', fmt='.2f', 
           cbar_kws={'label': 'Probabilidade de Salário Alto'}, 
           annot_kws={"size": 12})
plt.title('Probabilidade de Salário Alto por Formação Acadêmica e Experiência Profissional', fontsize=16)
plt.xlabel('Tempo de Experiência', fontsize=14)
plt.ylabel('Nível de Formação', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'interacao_formacao_experiencia.png'), 
           bbox_inches='tight', dpi=300)
plt.close()

# 8.8. NOVA VISUALIZAÇÃO: Importância das 3 principais features
top3_indices = indices[:3]
top3_features = [feature_names[i] for i in top3_indices]
top3_importances = importances[top3_indices]

# Criar gráfico de barras horizontais para as top 3 features
plt.figure(figsize=(10, 6))
plt.title('Top 3 Features Mais Importantes', fontsize=16)
bars = plt.barh(range(3), top3_importances, align='center', color=['#1f77b4', '#ff7f0e', '#2ca02c'])
plt.yticks(range(3), top3_features, fontsize=14)
plt.xlabel('Importância Relativa', fontsize=14)

# Adicionar os valores nas barras
for i, v in enumerate(top3_importances):
    plt.text(v + 0.01, i, f'{v:.4f}', va='center', fontsize=12)

plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'top3_features.png'), bbox_inches='tight', dpi=300)
plt.close()

# 8.9. NOVA VISUALIZAÇÃO: Gráfico de dispersão para as duas features mais importantes
if len(indices) >= 2:
    top2_indices = indices[:2]
    feature1 = feature_names[top2_indices[0]]
    feature2 = feature_names[top2_indices[1]]
    
    plt.figure(figsize=(12, 10))
    scatter = plt.scatter(X_test[feature1], X_test[feature2], 
                         c=y_pred_proba_test, cmap='coolwarm', 
                         alpha=0.7, s=100, edgecolors='k')
    
    plt.colorbar(scatter, label='Probabilidade de Salário Alto')
    plt.xlabel(feature1, fontsize=14)
    plt.ylabel(feature2, fontsize=14)
    plt.title(f'Relação entre as Duas Features Mais Importantes\n{feature1} vs {feature2}', fontsize=16)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'dispersao_top2_features.png'), bbox_inches='tight', dpi=300)
    plt.close()

print(f"\nTodos os gráficos foram salvos no diretório: {output_dir}")
